In [1]:
import nltk
from collections import defaultdict
import pandas as pd

In [2]:
from nltk.corpus import stopwords

In [3]:
from nltk.tokenize import TweetTokenizer

In [4]:
import fighting_words

In [39]:
import pickle

In [5]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/shengruilyu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [6]:
stop_words = set(stopwords.words('english'))
stop_words.add('removed')

In [8]:
df = pd.read_csv("../regex_filtering/after_regex_large.csv", index_col = 0, low_memory = False)

In [9]:
tknzr = TweetTokenizer()

In [10]:
word_frequency_by_subreddit= {}

In [11]:
# topic_postags = set(['NN', 'NNS', 'NNP', 'NNPS'])

In [12]:
tokens_by_subreddit = {}

In [13]:
posts_by_subreddit = {}

In [14]:
for index, row in df.iterrows():
    sub_reddit = row['subreddit']
    if sub_reddit in posts_by_subreddit:
        posts_by_subreddit[sub_reddit].append(row['body'])
    else:
        posts_by_subreddit[sub_reddit] = [row['body']]
#     tokens = tknzr.tokenize(row['body'])
#     if sub_reddit in tokens_by_subreddit:
#         tokens_by_subreddit[sub_reddit].append(tokens)
#     else:
#         tokens_by_subreddit[sub_reddit] = [tokens]
    
#     if sub_reddit in word_frequency_by_subreddit:
#         word_frequency = word_frequency_by_subreddit[sub_reddit]
#     else:
#         word_frequency = defaultdict(int)
#         word_frequency_by_subreddit[sub_reddit] = word_frequency
#     for token in tokens:
#         if (token not in stop_words) and (token.isalnum()):
#             word_frequency[token]+=1
    

In [15]:
subreddits = list(posts_by_subreddit.keys())

In [16]:
subreddits

['changemyview',
 'YouShouldKnow',
 'AskMen',
 'Futurology',
 'Art',
 'IAmA',
 'science',
 'Cooking']

In [17]:
topic_per_by_subreddit = {}
results_by_subreddit = {}

In [18]:
for current_subreddit in subreddits:
    l1 = posts_by_subreddit[current_subreddit]
    l2 = []
    for sr in subreddits:
        if sr == current_subreddit:
            print (sr)
            continue
        l2 += posts_by_subreddit[sr]
    results = fighting_words.bayes_compare_language(l1, l2)
    sort_results = sorted(results, key = lambda x : x[1], reverse = True)
    results_by_subreddit[current_subreddit] = sort_results
    

changemyview
Vocab size is 15000
Comparing language...
YouShouldKnow
Vocab size is 15000
Comparing language...
AskMen
Vocab size is 15000
Comparing language...
Futurology
Vocab size is 15000
Comparing language...
Art
Vocab size is 15000
Comparing language...
IAmA
Vocab size is 15000
Comparing language...
science
Vocab size is 15000
Comparing language...
Cooking
Vocab size is 15000
Comparing language...


In [20]:
duplicate = set([])
all_topics = set([])
for sr in results_by_subreddit.keys():
    for topic in results_by_subreddit[sr]:
        if topic in all_topics:
            duplicate.add(topic)
        all_topics.add(topic)

In [22]:
duplicate

set()

In [24]:
# remove from duplicate
# duplicate.remove('animals')
# duplicate.remove('cats')
# duplicate.remove('energy')
# duplicate.remove('meat')
# duplicate.remove('nuclear')
# duplicate.remove('virus')
# duplicate.remove('women')

In [25]:
duplicate

set()

In [26]:
remove_duplicate_topic = True

In [51]:
topk = 100

In [52]:
undesired_topic_postags = set(['IN'])

In [53]:
for current_subreddit in subreddits:
    topics = []
    sort_results = results_by_subreddit[current_subreddit]
    for word, z in sort_results:
        if len(topics) == topk:
            break
        if word in stop_words:
            continue
        if remove_duplicate_topic and word in duplicate:
            continue
        if nltk.pos_tag([word])[0][1] in undesired_topic_postags:
            continue
        topics.append(word)
    topic_per_by_subreddit[current_subreddit] = topics

In [54]:
topic_per_by_subreddit

{'changemyview': ['view',
  'trump',
  'biden',
  'vote',
  'comments',
  'rule',
  'op',
  'must',
  'argument',
  'explained',
  'comment',
  'delta',
  'breaking',
  'message',
  'page',
  'posted',
  'party',
  'review',
  'moderators',
  'appeal',
  'gender',
  'notice',
  'standards',
  'wiki',
  'ban',
  'gun',
  'note',
  'believe',
  'sexual',
  'link',
  'user',
  'arguments',
  'process',
  'voting',
  'change',
  'rights',
  'point',
  'information',
  'women',
  'person',
  'system',
  'nt',
  'favor',
  'argue',
  'agree',
  'lead',
  'trans',
  'evidence',
  'clicking',
  'rape',
  'multiple',
  'moral',
  'men',
  'crime',
  'bernie',
  'conversation',
  'democrats',
  'election',
  'rude',
  'would',
  'appeals',
  'stated',
  'court',
  'aspect',
  'voters',
  'gay',
  'definition',
  'example',
  'minor',
  'consent',
  'police',
  'president',
  'candidate',
  'falls',
  'willing',
  'assault',
  'moderation',
  'victim',
  'people',
  'progressive',
  'however',
  

In [55]:
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [56]:
save_obj(topic_per_by_subreddit, 'topics'+str(topk))

In [42]:
def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [43]:
loaded = load_obj('topics'+str(topk))

In [44]:
loaded

{'changemyview': ['view',
  'trump',
  'biden',
  'vote',
  'comments',
  'rule',
  'op',
  'must',
  'argument',
  'explained',
  'comment',
  'delta',
  'breaking',
  'message',
  'page',
  'posted',
  'party',
  'review',
  'moderators',
  'appeal'],
 'YouShouldKnow': ['dog',
  'flight',
  'refund',
  'bones',
  'cat',
  'ocd',
  'thoughts',
  'credit',
  'mail',
  'email',
  'card',
  'dogs',
  'vet',
  'bleach',
  'airline',
  'cancelled',
  'cancel',
  'pet',
  'dispute',
  'library'],
 'AskMen': ['relationship',
  'makeup',
  'men',
  'women',
  'girl',
  'girls',
  'friends',
  'dating',
  'date',
  'guys',
  'dad',
  'woman',
  'heavy',
  'girlfriend',
  'relationships',
  'partner',
  'attractive',
  'life',
  'look',
  'love'],
 'Futurology': ['money',
  'work',
  'ubi',
  'jobs',
  'income',
  'pay',
  'tax',
  'economy',
  'month',
  'people',
  'rich',
  'taxes',
  'companies',
  'nuclear',
  'coal',
  'energy',
  'job',
  'cost',
  'company',
  'rent'],
 'Art': ['art',
 

In [102]:
for index, row in df.iterrows():
    sub_reddit = row['subreddit']
    tokens = tknzr.tokenize(row['body'])
    new_tokens = []
    for token in tokens:
        if token in topic_per_by_subreddit[sub_reddit]:
            new_tokens.append("TOPIC")
        else:
            new_tokens.append(token)
    row['body'] = " ".join(new_tokens)

In [103]:
df.head()

,body,subreddit,author
0,I thought this was already the case ? Had it b...,changemyview,dublea
1,The original post had the moratorum lasting un...,changemyview,Ansuz07
2,"Technically , the H1N1 pandemic of 2009 was on...",changemyview,scott60561
3,The WHO declared an end of the H1N1 pandemic (...,changemyview,SaxonySam
4,Thank you for providing TOPIC with sources So ...,changemyview,epmuscle


In [104]:
if remove_duplicate_topic:
    filename = 'after_non_duplicate_top' + str(topk) + '_removal_large.csv'
else: 
    filename = 'after_top' + str(topk) + '_removal_large.csv'
df.to_csv(filename)